In [ ]:
import os
os.environ["PYTHONUTF8"] = "1"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from vnstock import Quote
from vnstock import Finance
from scipy import stats
import time
from typing import Literal
from scipy.interpolate import interp1d

<h1>I. Thu thap du lieu<h1>

<h2>1. Xac dinh cong ty can de tam<h2>
<h3>Phan nay khong can chay lai nua<h3>

In [ ]:
#Tim ma chung khoan cua cac cong ty
symbol = pd.read_csv('Du lieu.csv')
symbol.columns = symbol.iloc[0]
symbol = symbol.iloc[1::]
interest_industry = list(symbol['Lĩnh vực'].unique())
interest_industry.pop(1)
symbol = symbol[symbol['Lĩnh vực'].isin(interest_industry)]
symbol_name = list(symbol['Công ty'])

In [ ]:
#Phan chia so lan lay du lieu
STT_list = [0,55,110,156]
name_bins = {}
for i in range(len(STT_list)):
    if STT_list[i] == 0:
        continue
    else:
        name_bins[STT_list[i]] = symbol_name[STT_list[i-1]:STT_list[i]]
history_price = pd.DataFrame()

In [ ]:
#Thu thap du lieu ve gia
#Khong can chay o du lieu nay
for i in name_bins[156]:
    pr = Quote(source='VCI',symbol=i)
    price_status = pr.history(start='2010-01-01',end='2025-01-01',interval='1M')
    price_status['time'] = pd.to_datetime(price_status['time'])
    price_status.set_index('time', inplace=True)
    history_price[i] = price_status['close']

<h2>2. Chon loc ra cac co phieu trong so 200 cp co gia tri von hoa lon nhat<h2>
<h3>Phan nay khong can chay lai nua<h3>

In [ ]:
#chia co phieu ra lam 2 thoi ky
history_price = pd.read_csv('price data.csv')
history_price['time'] = pd.to_datetime(history_price['time'])
history_price.set_index('time', inplace=True)
history_price = history_price[history_price.index.year>2015]
history_price.dropna(axis=1,inplace=True)
history_price.to_csv('compare_period1.csv')


<h2>3. Tai du lieu BCTC ve<h2>
<h3>Phan nay khong can chay lai nua<h3>

In [ ]:
#Cac co phieu can lay bao cao tai chinh
data = pd.read_csv('compare_period1.csv')
data['time'] = pd.to_datetime(data['time'])
data.set_index('time',inplace=True)
stock_of_interest = list(data.columns)

In [ ]:
#chia ra thanh cac muc nho hon de lay du lieu
count = 0
stock_trench = {}
number_stock_per_trench = 7
number_of_trench = int(round(len(stock_of_interest)/number_stock_per_trench,0)+1)
separate_support = [min(number_stock_per_trench*i, len(stock_of_interest)) for i in range(number_of_trench)]#cu chon  co phieu 1 danh sach
for i in range(1,len(separate_support)):
    stock_trench[separate_support[i]] = stock_of_interest[separate_support[i-1]:separate_support[i]]
financial_statement_data = pd.DataFrame()#phuc vu cho cell ke tiep
stock_trench

In [ ]:
char_cf = ['Năm', 'Kỳ','Khấu hao TSCĐ',
           'Mua sắm TSCĐ','Tiền thu được từ thanh lý tài sản cố định'  ]
char_is = ['Năm', 'Kỳ','Doanh thu thuần','Thu nhập tài chính','Thu nhập khác','LN trước thuế','Chi phí tiền lãi vay']
char_bs = ['Năm', 'Kỳ','TÀI SẢN NGẮN HẠN (đồng)','Tiền và tương đương tiền (đồng)', 'Giá trị thuần đầu tư ngắn hạn (đồng)','Nợ ngắn hạn (đồng)',
           'Vay và nợ thuê tài chính ngắn hạn (đồng)',
           'Vay và nợ thuê tài chính dài hạn (đồng)',
           'TỔNG CỘNG TÀI SẢN (đồng)','VỐN CHỦ SỞ HỮU (đồng)','LỢI ÍCH CỦA CỔ ĐÔNG THIỂU SỐ']
char_ratio = ['Năm', 'Kỳ','Số CP lưu hành (Triệu CP)']
sleep_time = 40
for t in list(stock_trench.keys())[-2::]:
    for i in stock_trench[t]:
        if i == "DPP":
            continue
        else:
            finance = Finance(source='VCI',symbol=i)
            #bao cao ket qua hoat dong kinh doanh
            income_state = finance.income_statement(lang='vi')
            for j in char_is:
                if j not in income_state.columns:
                    income_state[j]=0
            income_state = income_state[char_is]
            income_state = income_state[income_state['Năm']>2012]
            year = list(income_state['Năm'])
            quarter = list(income_state['Kỳ'])
            time1 = [f'{i}Q{j}' for i,j in zip(year,quarter)]
            time1 = pd.PeriodIndex(time1,freq='Q').to_timestamp(how='end').normalize()
            income_state['time'] = time1
            income_state.set_index('time',inplace=True)
            income_state.drop(['Năm','Kỳ'],axis=1, inplace=True)
            #bang can doi ke toan
            balance_sheet = finance.balance_sheet(lang='vi')
            for j in char_bs:
                if j not in balance_sheet.columns:
                    balance_sheet[j]=0
            balance_sheet = balance_sheet[char_bs]
            balance_sheet = balance_sheet[balance_sheet['Năm']>2012]
            balance_sheet['time'] = time1
            balance_sheet.drop(['Năm','Kỳ'],axis=1, inplace=True)
            balance_sheet.set_index('time', inplace=True)
            #bao cao luu chuyen tien te
            cashflow = finance.cash_flow(lang='vi')
            for j in char_cf:
                if j not in cashflow.columns:
                    cashflow[j] = 0
            cashflow = cashflow[char_cf]
            cashflow = cashflow[cashflow['Năm']>2012]
            cashflow['time'] = time1
            cashflow.set_index('time', inplace=True)
            cashflow.drop(['Năm','Kỳ'],axis=1, inplace=True)
            #so luong co phieu dang luu hanh
            outstanding_share = finance.ratio(lang='vi')
            outstanding_share = outstanding_share.droplevel(level=0,axis=1)
            for j in char_ratio:
                if j not in outstanding_share.columns:
                    outstanding_share[j] = 0
            outstanding_share = outstanding_share[char_ratio]
            outstanding_share = outstanding_share[outstanding_share['Năm']>2012]
            outstanding_share['time']=time1
            outstanding_share.set_index('time',inplace=True)
            outstanding_share.drop(['Năm','Kỳ'],axis=1, inplace=True)
            #sap nhap cac bao cao lai thanh 1 dataframe
            financial_statement_subdata = pd.concat([income_state,balance_sheet,cashflow,outstanding_share], axis=1)
            financial_statement_subdata['ticket'] = i
            #sap nhap cac cong ty lai thanh 1
            if len(financial_statement_data) == 0:
                financial_statement_data = financial_statement_subdata
            else:
                financial_statement_data = pd.concat([financial_statement_data,financial_statement_subdata],axis=0)
    time.sleep(sleep_time)


<h2>4. Xac nhan lai cac cong ty can de tam, dong thoi thu thap du lieu gia<h2>
<h3>Phan nay khong can chay lai nua<h3>

In [ ]:
#lay du lieu
financial_statement_data = pd.read_csv('financial_statements.csv')
financial_statement_data['time'] = pd.to_datetime(financial_statement_data['time'])
financial_statement_data.set_index(['ticket','time'], inplace=True)

In [ ]:
#Phan trench de lay du lieu
stock_of_interest = list(financial_statement_data.index.get_level_values(0).unique())
stock_trench = {}
number_stock_per_trench = 20
number_of_trench = int(round(len(stock_of_interest)/number_stock_per_trench,0)+1)
trench = [min(number_stock_per_trench*i,len(stock_of_interest)) for i in range(number_of_trench)]
l=0
for i in range(1,len(trench)):
    stock_trench[trench[i]] = stock_of_interest[trench[i-1]:trench[i]]
    l += len(stock_of_interest[trench[i-1]:trench[i]])



In [ ]:
input_data = pd.DataFrame()
compare_data = pd.DataFrame()
for t in stock_trench.keys():
    for i in stock_trench[t]:
        his = Quote(source='VCI', symbol=i)
        #du lieu dau vao
        input_subdata = his.history(start='2015-01-01',end='2026-01-01',interval='1M')
        input_subdata['time'] = pd.to_datetime(input_subdata['time'])
        input_subdata.set_index('time',inplace=True)
        input_subdata = input_subdata[['close']]
        input_subdata.columns = [i]
        #Du lieu so sanh
        compare_subdata = his.history(start='2020-01-01',end='2026-01-01',interval='1D')
        compare_subdata['time'] = pd.to_datetime(compare_subdata['time'])
        compare_subdata.set_index('time',inplace=True)
        compare_subdata = compare_subdata[['close']]
        compare_subdata.columns = [i]
        if len(input_data)==0:
            input_data = input_subdata
        else:
            input_data = pd.concat([input_data,input_subdata],axis=1)
        if len(compare_data)==0:
            compare_data = compare_subdata
        else:
            compare_data = pd.concat([compare_data,compare_subdata],axis=1)
    time.sleep(35)
    

In [ ]:
input_data = input_data.dropna(axis=0)
compare_data = compare_data[compare_data.index.year>2020]
input_data.to_csv('input_data.csv')
compare_data.to_csv('compare_data.csv')

In [ ]:
ticket_general_info = pd.read_csv('Du_lieu.csv')
col = ticket_general_info.iloc[0].to_list()
ticket_general_info.columns = col
ticket_general_info = ticket_general_info.drop(index=ticket_general_info.index[0])
ticket_general_info.set_index('Công ty',inplace=True)

In [ ]:
ticket_of_interest = pd.read_csv('compare_data.csv')
ticket_of_interest['time'] = pd.to_datetime(ticket_of_interest['time'])
ticket_of_interest.set_index('time',inplace=True)
info_of_interest_ticket = ticket_general_info.loc[ticket_of_interest.columns]
info_of_interest_ticket.to_csv('info_of_ticket.csv')

<h1>II. Thuat toan phan bo danh muc<h1>

<h2>1. Xay dung du lieu dau vao cho mo hinh<h2>

<h3>1.1. Bo chi tieu dau vao va cac thanh phan phu khac <h3>

In [ ]:
#xu ly du lieu dau vao
raw_data = pd.read_csv('financial_statements.csv')
raw_data['time']= pd.to_datetime(raw_data['time'])
raw_data.set_index(['ticket','time'],inplace=True)
raw_data = raw_data.sort_index(level='time',ascending=True)

In [ ]:
#Tinh cac chi tieu trung gian 
intermediary_data = raw_data['Doanh thu thuần']+raw_data['Thu nhập tài chính']+raw_data['Thu nhập khác']
intermediary_data = intermediary_data.to_frame(name='dttn')
## chi phí được ghi âm thay vì dương nên cần phải cận thận khi công trừ các chi tiêu này
intermediary_data['ebit'] = raw_data['LN trước thuế'] - raw_data['Chi phí tiền lãi vay']
intermediary_data['dna'] = raw_data['Khấu hao TSCĐ']
intermediary_data['capex'] = -raw_data['Mua sắm TSCĐ'] - raw_data['Tiền thu được từ thanh lý tài sản cố định']
intermediary_data['wc'] = (raw_data['TÀI SẢN NGẮN HẠN (đồng)']-raw_data['Giá trị thuần đầu tư ngắn hạn (đồng)'] - raw_data[
    'Tiền và tương đương tiền (đồng)']) - (raw_data['Nợ ngắn hạn (đồng)']-raw_data['Vay và nợ thuê tài chính ngắn hạn (đồng)'])
intermediary_data['debt'] = raw_data['Vay và nợ thuê tài chính ngắn hạn (đồng)']+raw_data['Vay và nợ thuê tài chính dài hạn (đồng)']
intermediary_data['equity'] = raw_data['VỐN CHỦ SỞ HỮU (đồng)']
intermediary_data['value'] = intermediary_data['debt']+intermediary_data['equity']#day chinh la chi tieu V
intermediary_data['TA'] = raw_data['TỔNG CỘNG TÀI SẢN (đồng)']
#Tao dung cac mau du lieu dau tien, dữ liệu này là kỳ gần nhất đóng vai trò tính toán ra các dữ liệu trong tương lai ví dụ nhu DTTN
latest_time_of_raw_data = raw_data.groupby(level='ticket').last()
latest_time_of_intermediary_data =intermediary_data.groupby(level='ticket').last() 
ind = latest_time_of_raw_data.index
col = ['dttn','wc','minor_rate','out_share']
initial_data_set = np.full(shape=(len(ind),len(col)),fill_value=0)
initial_data_set = pd.DataFrame(initial_data_set,columns=col,index=ind)
initial_data_set['dttn'] = latest_time_of_intermediary_data['dttn']
initial_data_set['wc'] = latest_time_of_intermediary_data['wc']
initial_data_set['minor_rate'] = latest_time_of_raw_data['LỢI ÍCH CỦA CỔ ĐÔNG THIỂU SỐ']/latest_time_of_raw_data['VỐN CHỦ SỞ HỮU (đồng)']
initial_data_set['out_share'] = latest_time_of_raw_data['Số CP lưu hành (Triệu CP)']

In [ ]:
#Bo chi tieu can quan tam
input_ratio = np.log(intermediary_data['dttn']/intermediary_data['dttn'].groupby(level='ticket').shift())
input_ratio = input_ratio.to_frame(name='g_log')
input_ratio['omr'] = intermediary_data['ebit']/intermediary_data['dttn']
input_ratio['dpoc'] = intermediary_data['dna']/(intermediary_data['dttn']-intermediary_data['ebit'])
input_ratio['lrr'] = intermediary_data['dna']/intermediary_data['capex']
input_ratio['wct'] = intermediary_data['dttn']/intermediary_data['wc']
input_ratio['hskd'] = intermediary_data['dttn']/intermediary_data['TA']
input_ratio['ht'] = intermediary_data['equity']/intermediary_data['TA']
input_ratio['wod'] = intermediary_data['debt']/(intermediary_data['TA']-intermediary_data['equity']) 

<h2>2. Xay dung cau truc phu thuoc giua cac bien<h2>

<h3>2.1. Chuan hoa bo chi tieu<h3>

<h4>2.1.1. Cac ham lien quan<h4>

In [ ]:
#Xay dung ham phan phoi dua tren gaussian kde
data_type = Literal['in_R','minus_infty_to_one','binary','zero_to_infty']
infinitesimal = 1e-9
quantity_for_intepld = 1000000
class kde_distribution:
    def __init__(self,data_set,type_of_data:data_type):
        self.type = type_of_data
        data = data_set.replace([np.inf,-np.inf],np.nan)
        data = data_set.dropna()
        if self.type == 'in_R':
            self.data = data
            self.range = np.linspace(-200,200,quantity_for_intepld)
        elif self.type == 'minus_infty_to_one':
            self.data = -np.log(1-data)
            self.range = np.linspace(-200,1-infinitesimal,quantity_for_intepld)
        elif self.type == 'binary':
            data += infinitesimal
            self.data = np.log(data/(1-data))
            self.range = np.linspace(0+infinitesimal,1-infinitesimal,quantity_for_intepld)
        elif self.type == 'zero_to_infty':
            self.data = np.log(data)
            self.range = np.linspace(0+infinitesimal,200,quantity_for_intepld)
        self.kde = stats.gaussian_kde(self.data)
        self._pdf = self.pdf(self.range)
        self.cdf_val = np.cumsum(self._pdf)
        self.cdf_val/= self.cdf_val[-1]
        self._cdf = interp1d(self.range,self.cdf_val,kind='linear',bounds_error=False,fill_value=(0,1))
        self._ppf = interp1d(self.cdf_val,self.range,kind='linear',bounds_error=False, fill_value="extrapolate")
    def pdf(self, x):
        if self.type == 'in_R':
           p = self.kde(x)
        elif self.type == 'minus_infty_to_one':
            p = self.kde(-np.log(1-x))/np.log(1-x) 
        elif self.type == 'binary':
            p = self.kde(np.log(x/(1-x)))/(x-x**2)
        elif self.type == 'zero_to_infty':
            p = self.kde(np.log(x))/x
        return p
    def cdf (self,x):
        return self._cdf(x)
    def ppf (self,u):
        return self._ppf(u)
        


<h4>2.1.2. Thuat toan normalize chi tieu bang CDF<h4>

In [ ]:
#Phan nhom bo chi tieu
in_R = ['g_log','lrr','wct']
minus_infty_to_one = ['omr','dpoc']
binary = ['ht','wod']
zero_to_infty = ['hskd']
input_ratio = input_ratio.replace([-np.inf,np.inf],np.nan)
ticket = input_ratio.index.get_level_values(level=0).unique()
ratio = input_ratio.columns
standardised_input_ratio = pd.DataFrame()
#Chuan hoa du lieu
for i in ticket:
    data_set = input_ratio.xs(i,level=0)
    single_stock = pd.DataFrame()
    for j in ratio:
        n = len(data_set[j].dropna())+1
        if len(single_stock) == 0:
            single_stock = data_set[[j]].rank(method='average')/n
        else:
            mediate = data_set[[j]].rank(method='average')/n
            single_stock = pd.concat([single_stock,mediate],axis=1)
    single_stock['ticket'] = i
    if len(standardised_input_ratio)==0:
        standardised_input_ratio = single_stock
    else:
        standardised_input_ratio = pd.concat([standardised_input_ratio,single_stock],axis=0)
standardised_input_ratio.set_index('ticket',append=True,inplace=True)
standardised_input_ratio.reorder_levels(['ticket','time'],axis=0)
    

<h3>2.2. Xay dung thu tu phu thuoc<h3>

<h4>2.2.1. Su phu thuoc giua cac doanh nghiep voi nhau<h4>